1. 词向量我用了github下载的词向量, 是cboe的词向量, 我觉得他们传到github的用的文本数据应该比我的多, 训练的也比我的好, 类似的还有google自己训练的英文的词向量, 我觉得不应该用自己训练的

2. 有些词不在词向量的我把词再做了一次切割, 如果在词向量, 就用切割变成的两个词, 如果还是不在就舍弃, 防止了'网银盾'这样的词语一开始不在词向量中

3. 用了sif做句向量, 另外保留了sif制作过程中的pca的第一个轴(u), 在后面输入一句话的时候, 我认为也需要去除这个主要轴的影响

4. 我用了Biopython的余弦距离的聚类, 进行四分类, 一类是业务的问答, 三类是非业务的问答, 感觉正确率在90%以上, 但是biopython没有返回聚类的点或者轴的位置, 所以我用一个类的所有样本的句向量的加和作为了轴的位置

5. 在快速获取句向量的时候, 我用了递归做了一个句向量的索引, 先把全部的句向量做一个余弦距离的无监督聚类, 然后如果聚类的数量超过1024, 我就用这个聚类过的一个类别再进行4个cluster的聚类, 直到每一个类别都少于1024个句向量. 在获取的时候, 我用句向量和4个聚类的轴计算余弦距离, 取最近的余弦距离的轴, 在句向量的索引中通过这个聚类轴作为键, 找到值, 如果值还是一个dict, 就继续上一步的操作, 直到值是一个列表, 包含了所有类似的不多于1024个的句向量, 取前10个句向量

6. 我建的句向量的索引查最适合的10个句子, 发现耗时是0.5秒, 还不是很理想, 布尔搜索是0.1秒, 还可以. 在排序上面如果用c语言的堆排, 取前10个可以快一些, 另外布尔搜索时, Python的sort是稳定排序, 容易出现数据库中的数据和输入的句子相比, 有不同的共同词语, 但是第一个共同词语数量超过10个, 所以结果都是第一个词语的, 通过shuffle解决了

7. 取最终的一个词语时, 如果这个回复在向量搜索中和布尔搜索中都有, 就用这个, 如果没有, 就用向量搜索的第一个

8. 最终判断是否是业务问题的准确率precision大约是97%, recall接近100%

9. 爬百度会很麻烦, 我之前爬过google, 我用selenium爬的, 开selenium延迟会很高, 不适合, 而且google有付费的搜索接口, 不需要爬虫